In [1]:
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import KFold, cross_val_score
import pandas as pd
import sklearn.linear_model as lm
from sklearn.feature_selection import RFE, SelectKBest
from sklearn.model_selection import train_test_split
from itertools import combinations
import matplotlib.pyplot as plt
import pylab
from sklearn import metrics
import math
import imp


In [2]:
metricsUtil = imp.load_source('MetricsUtil', 'MetricsUtil.py')

data = pd.read_csv('Dataset\Hitters.csv', usecols=range(0,21), parse_dates=True).dropna()

data.head()

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N
5,-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,...,501,336,194,A,W,282,421,25,750.0,A


In [3]:
def process_subset(feature_set, x_train, y_train, x_test, y_test ):
    d = len(feature_set)
    n = y_test.shape[0]
    # Fit model on feature_set  
    model = lm.LinearRegression().fit(x_train[[i for i in feature_set]], y_train)
    Y_hat = model.predict(x_test[list(feature_set)])
    
    rss = metricsUtil.RSS(y_test, Y_hat)  

    #mse = metrics.mean_squared_error(y_test, Y_hat)
    mse = metricsUtil.Mse(y_test, Y_hat)
    
    return {"model":model, 
            "RSS":rss,
            "features": feature_set,
            "mse": mse }


In [4]:
def best_subset_selection(k, x_train, y_train, x_test, y_test ):
    results = []
    for combo in combinations(x_train.columns, k):
        results.append(process_subset(combo, x_train, y_train, x_test, y_test))
    models = pd.DataFrame(results)
    #Choose best model based on RSS
    best_model = models.loc[models['RSS'].argmin()]
    return best_model

subsets = pd.DataFrame(columns=["RSS", "model", "features", "mse"])


In [5]:
# ********* Validation split ************
train, test = train_test_split(data, test_size = 0.5)

X_train = pd.get_dummies(train.drop('Salary', axis=1).drop('Unnamed: 0', axis=1)).drop(["League_A", "Division_E", "NewLeague_A"], axis=1)
Y_train = train['Salary']

X_test = pd.get_dummies(test.drop('Salary', axis=1).drop('Unnamed: 0', axis=1)).drop(["League_A", "Division_E", "NewLeague_A"], axis=1)
Y_test = test['Salary']

for i in range(1, 20):
    subsets.loc[i] = best_subset_selection(i, X_train, Y_train, X_test, Y_test)
    


C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is depre

In [6]:
# ***** Print  features and mse *****
RSS = subsets.RSS
features = subsets.features
mse = subsets.mse
print("features")
print( features)
print( "mse")
print( mse)
print( "rss")
print( RSS)

features
1                                               (CRBI,)
2                                       (CRBI, PutOuts)
3                                (Years, CRBI, PutOuts)
4                      (CAtBat, CHits, CHmRun, PutOuts)
5          (CAtBat, CHits, CHmRun, PutOuts, Division_W)
6     (CAtBat, CHits, CHmRun, PutOuts, Assists, Divi...
7     (CAtBat, CHits, CHmRun, PutOuts, Assists, Erro...
8     (Years, CAtBat, CHits, CHmRun, PutOuts, Assist...
9     (Years, CAtBat, CHits, CHmRun, CRBI, PutOuts, ...
10    (Years, CAtBat, CHits, CHmRun, CRBI, PutOuts, ...
11    (HmRun, RBI, CAtBat, CHits, CHmRun, CRBI, PutO...
12    (HmRun, RBI, CAtBat, CHits, CHmRun, CRBI, PutO...
13    (HmRun, RBI, Years, CAtBat, CHits, CHmRun, CRB...
14    (HmRun, RBI, Years, CAtBat, CHits, CRuns, CRBI...
15    (HmRun, RBI, Years, CAtBat, CHits, CHmRun, CRu...
16    (AtBat, Hits, HmRun, Walks, Years, CAtBat, CHi...
17    (AtBat, Hits, HmRun, RBI, Walks, Years, CAtBat...
18    (AtBat, Hits, HmRun, Runs, RBI, W

In [7]:
# *****   Cross validation *******
X = pd.get_dummies(data.drop('Salary', axis=1).drop('Unnamed: 0', axis=1)).drop(["League_A", "Division_E", "NewLeague_A"], axis=1)
Y = data["Salary"]

kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(X)
j = 0
mse = np.zeros((kf.n_splits, 19))
for train_index, test_index in kf.split(X):
    print( "run")
    X_train = X.iloc[train_index]
    Y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_test = Y.iloc[test_index]
    for i in range(1, 20):
         subset = best_subset_selection(i, X_train, Y_train, X_test, Y_test)
         mse[j,i-1] = subset["mse"]
    j = j+1
print(mse.mean(axis=0))    


run


C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is depre

run


C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is depre

run


...

C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is depre

run


C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is depre

[127672.14177173 100838.25259391  92417.73822707  86769.7149317
  83444.94223882  82324.69149973  81787.17439781  81607.36894396
  81361.13658528  81470.94078763  81979.77524609  83255.2288922
  85296.33864534  86936.58079994  89937.32220363  93617.42579065
  97297.09470877 103028.46073742 113096.44805942]


C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys
C:\Users\ida_s\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys


In [8]:
# ***** Print  features and mse *****
RSS = subsets.RSS
features = subsets.features
mse = subsets.mse
print("features")
print( features)
print( "mse")
print( mse)
print( "rss")
print( RSS)

features
1                                               (CRBI,)
2                                       (CRBI, PutOuts)
3                                (Years, CRBI, PutOuts)
4                      (CAtBat, CHits, CHmRun, PutOuts)
5          (CAtBat, CHits, CHmRun, PutOuts, Division_W)
6     (CAtBat, CHits, CHmRun, PutOuts, Assists, Divi...
7     (CAtBat, CHits, CHmRun, PutOuts, Assists, Erro...
8     (Years, CAtBat, CHits, CHmRun, PutOuts, Assist...
9     (Years, CAtBat, CHits, CHmRun, CRBI, PutOuts, ...
10    (Years, CAtBat, CHits, CHmRun, CRBI, PutOuts, ...
11    (HmRun, RBI, CAtBat, CHits, CHmRun, CRBI, PutO...
12    (HmRun, RBI, CAtBat, CHits, CHmRun, CRBI, PutO...
13    (HmRun, RBI, Years, CAtBat, CHits, CHmRun, CRB...
14    (HmRun, RBI, Years, CAtBat, CHits, CRuns, CRBI...
15    (HmRun, RBI, Years, CAtBat, CHits, CHmRun, CRu...
16    (AtBat, Hits, HmRun, Walks, Years, CAtBat, CHi...
17    (AtBat, Hits, HmRun, RBI, Walks, Years, CAtBat...
18    (AtBat, Hits, HmRun, Runs, RBI, W